In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm
import datetime
import glob
import logic.pulsed.pulse_objects as po
import scipy.signal

### Attempt at rabi analysis

In [ ]:
locs = 'C:/Data/Prime95B/2020/11/20201126/Pulsed/auto_rabi_small_roi_2000_0_-20_500_(150, 138)pxs_-2dBm_20201127-0856-50.npz'

data = np.load(locs)
a = data['data'][:,0,40:52,80:82]/data['data'][:,1,40:52,80:82]
y = np.mean(a, axis=(1,2))
x = np.arange(2000,0,-20)*1e-9*1e6
plt.plot(x, y, 'ro')

fit = fitlogic.make_sinetriplewiththreeexpdecay_fit(x_axis=x/1e6, data=y, estimator=fitlogic.estimate_sinetriplewiththreeexpdecay)
plt.plot(x, fit.best_fit, 'b--')
plt.show()
print(fit.fit_report())


### Attempt at T1 analysis

In [ ]:
locs = 'C:/Data/Prime95B/2020/11/20201127/auto_T2_small_roi_6.91e6Hz_wref_pih_201000_0_-5000_100_(150, 138)pxs_-2dBm_20201129-2215-27.npz'

data = np.load(locs)
a = 1/(data['data'][:,0,:,:])*(data['data'][:,1,:,:])
y = np.mean(a, axis=(1,2))
x = np.arange(201000,0,-5000)*1e-9
plt.plot(x, y, 'ro-')

# fit = fitlogic.make_decayexponentialstretched_fit(x_axis=np.flip(x, axis=0), data=np.flip(y, axis=0), estimator=fitlogic.estimate_decayexponentialstretched)
# plt.plot(np.flip(x, axis=0), fit.best_fit, 'b--')
# plt.show()
# # print(fit.fit_report())#, fit.params['frequency']/1e6, 1/fit.params['frequency'])
# plt.plot(x, np.mean(data['data'][:,0,:,:], axis=(1,2)),'bo-')
# plt.plot(x, np.mean(data['data'][:,1,:,:], axis=(1,2)), 'ro-')
# plt.show()

locs = 'C:/Data/Prime95B/2020/11/20201127/auto_T2_small_roi_6.91e6Hz_wref_3pih_201000_0_-5000_100_(150, 138)pxs_-2dBm_20201129-2314-22.npz'

data = np.load(locs)
a = 1/(data['data'][:,0,:,:])*(data['data'][:,1,:,:])
z = np.mean(a, axis=(1,2))
x = np.arange(201000,0,-5000)*1e-9
plt.plot(x, z, 'ro-')
plt.plot(x, y, 'bo-')

# fit = fitlogic.make_decayexponentialstretched_fit(x_axis=np.flip(x, axis=0), data=np.flip(y-z, axis=0), estimator=fitlogic.estimate_decayexponentialstretched)
# plt.plot(np.flip(x, axis=0), fit.best_fit, 'b--')
plt.show()
# print(fit.fit_report())#, fit.params['frequency']/1e6, 1/fit.params['frequency'])
plt.plot(x, np.mean(data['data'][:,0,:,:], axis=(1,2)),'bo-')
plt.plot(x, np.mean(data['data'][:,1,:,:], axis=(1,2)), 'ro-')
plt.show()

## For old data

In [ ]:
av = []
x = []
locs = sorted(glob.glob('C:/Data/Prime95B/2020/11/20201124/Pulsed/*run_8*'), key=len)
for i,loc in enumerate(locs):
    data = np.load(loc)
#     print(loc)
#     a = np.mean(data['data'], axis=(1,2))
    a = data['data'][0]/data['data'][1]
    a = np.mean(a[:,:])
    av.append(a)
    x.append(i)
x = np.arange(10,1200,20)*1e-9
y = np.array(av)
plt.plot(x, y, 'ro')

fit = fitlogic.make_sineexponentialdecay_fit(x_axis=x[:], data=y[:], estimator=fitlogic.estimate_sineexponentialdecay)
plt.plot(x[:], fit.best_fit, 'b--')
plt.show()
print(fit.fit_report(), fit.params['frequency']/1e6, 1/fit.params['frequency'])


In [ ]:
def determine_ideal_mw_power(rabi_frequency, p0, v1):
    p1 = p0 - 20*np.log10(rabi_frequency/v1) 
    return round(p1,2)
print(determine_ideal_mw_power(11.7*1e6, -6, 1/(40e-9)))

In [ ]:
t = np.arange(5,100,2)*1e-9
av = np.array(av)
sp = np.fft.fft(av)/len(av)
sp = sp[range(int(len(av)/2))]
samplingFrequency = len(t)
tpCount     = len(av)
values      = np.arange(int(tpCount/2))
timePeriod  = tpCount/samplingFrequency
frequencies = values/timePeriod

# freq = np.fft.fftfreq(t.shape[-1])
plt.plot(frequencies, abs(sp), 'bo')#, freq, sp.imag)
plt.show()
print(frequencies[0])

# Creating pulse ensemble, uploading, running, collecting images - loop and save

# ODMR

In [45]:
def make_odmr_pulse_block(ns, not_ref, rabi_ns):
    ele = []
    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': not_ref, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=rabi_ns/2*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

    d_ch = {'d_ch1': True, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=3e-6, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=500e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=1e-6, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    pulse_block = po.PulseBlock(name=f'auto_odmr_P95B_{ns}_{not_ref}', element_list=ele)
    
    return pulse_block

# Rabi

In [10]:
def make_rabi_pulse_block(ns, not_ref, rabi_ns=None):
    ele = []
    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': not_ref, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=ns*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

    d_ch = {'d_ch1': True, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=3e-6, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=500e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=1e-6, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    pulse_block = po.PulseBlock(name=f'auto_rabi_P95B_{ns}_{not_ref}', element_list=ele)
    
    return pulse_block

# T1

In [3]:
def make_T1_pulse_block(ns, not_ref, rabi_ns):
    ele = []
    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': not_ref, 'd_ch3': False}
    ele.append(po.PulseBlockElement(init_length_s=rabi_ns/2*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False}
    ele.append(po.PulseBlockElement(init_length_s=ns*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

    d_ch = {'d_ch1': True, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False}
    ele.append(po.PulseBlockElement(init_length_s=3e-6, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False}
    ele.append(po.PulseBlockElement(init_length_s=500e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False}
    ele.append(po.PulseBlockElement(init_length_s=1e-6, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    pulse_block = po.PulseBlock(name=f'auto_T1_P95B_{ns}_{not_ref}', element_list=ele)
    
    return pulse_block

# Hahn Echo

In [2]:
def make_HEcho_pulse_block(ns, not_ref, rabi_ns):
    if not_ref:
        last_ns = rabi_ns/4
    else:
        last_ns = rabi_ns*3/4
    ele = []
    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': True, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=rabi_ns/4*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=ns/2*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': True, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=rabi_ns/2*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=ns/2*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': True, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=last_ns*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    d_ch = {'d_ch1': True, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=3e-6, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=500e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=1e-6, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    pulse_block = po.PulseBlock(name=f'auto_HEcho_P95B_{ns}_{not_ref}', element_list=ele)
    
    return pulse_block

## Hahn Echo w/ reference

In [ ]:
def make_HEcho_pulse_block_w_ref_pih(ns, not_ref, rabi_ns):
    ele = []
    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': not_ref, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=rabi_ns/4*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=ns/2*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': not_ref, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=rabi_ns/2*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=ns/2*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': not_ref, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=rabi_ns/4*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    d_ch = {'d_ch1': True, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=3e-6, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=500e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=1e-6, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    pulse_block = po.PulseBlock(name=f'auto_HEcho_P95B_{ns}_{not_ref}', element_list=ele)
    
    return pulse_block

def make_HEcho_pulse_block_w_ref_3pih(ns, not_ref, rabi_ns):
    ele = []
    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': not_ref, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=rabi_ns/4*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=ns/2*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': not_ref, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=rabi_ns/2*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=ns/2*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': not_ref, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=rabi_ns*3/4*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    d_ch = {'d_ch1': True, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=3e-6, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=500e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
    ele.append(po.PulseBlockElement(init_length_s=1e-6, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    pulse_block = po.PulseBlock(name=f'auto_HEcho_P95B_{ns}_{not_ref}', element_list=ele)
    
    return pulse_block

### Create and upload pulse ensemble

In [6]:
def make_trigger_cam_block(ms, odmr=False):
    ele = []
    d_ch = {'d_ch1': False, 'd_ch2': True, 'd_ch4': False, 'd_ch3': odmr}
    ele.append(po.PulseBlockElement(init_length_s=ms*1e-3, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    pulse_block = po.PulseBlock(name=f'trigger_cam_{odmr}_{ms}', element_list=ele)
    
    return pulse_block

def wait_block(ms):
    ele = []
    d_ch = {'d_ch1': False, 'd_ch2': False, 'd_ch4': False, 'd_ch3': False}
    ele.append(po.PulseBlockElement(init_length_s=ms*1e-3, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
    
    pulse_block = po.PulseBlock(name=f'wait_block_{ms}', element_list=ele)
    
    return pulse_block

### Upload function

In [7]:
def upload_CW_ensemble(t_ns, rep, pulse_fnt, rabi_ns, odmr):
    pulse = pulse_fnt(t_ns, True, rabi_ns)
    pulse_ref = pulse_fnt(t_ns, False, rabi_ns)
    trigger_cam_1 = make_trigger_cam_block(26, odmr)
    trigger_cam_2 = make_trigger_cam_block(26)
    wait = wait_block(100)

    created_blocks = [pulse, pulse_ref, trigger_cam_1, trigger_cam_2, wait]
    for block in created_blocks:
        sequencegeneratorlogic.save_block(block)

    bl = [(trigger_cam_1.name,1), (pulse.name,rep), (wait.name,1), (trigger_cam_2.name,1), (pulse_ref.name,rep)]
    auto_pulse_CW = po.PulseBlockEnsemble(name='auto_pulse_CW', block_list=bl, rotating_frame=False)

    ensemble = auto_pulse_CW
    ensemblename = auto_pulse_CW.name
    sequencegeneratorlogic.save_ensemble(ensemble)
    ret,_,_ = sequencegeneratorlogic.sample_pulse_block_ensemble(ensemblename)
    sequencegeneratorlogic.load_ensemble(ensemblename)
    return ret

### Run pulse and collect image function

In [8]:
def do_cam_CW_pulsed_measure():
    mycamera.ready_pulsed(2)

    pulsedmeasurementlogic.pulse_generator_on()
    mycamlogic.start_trigger_seq(2)
    
    pulsedmeasurementlogic.pulse_generator_off()
    return mycamlogic.get_last_image()

def set_start_MW(power, freq):
    settings = {
    'power': power,
    'frequency': freq,
    'use_ext_microwave': True
    }
    pulsedmeasurementlogic.set_microwave_settings(settings_dict=settings)
    pulsedmeasurementlogic.microwave_on()

### Main function

In [9]:
def main(start, stop, step, n_runs, rep, power, freq, tag, pulse_fnt, rabi_ns, odmr=False):
    sequencegeneratorlogic.pulse_generator_settings = {'activation_config':'pulsed_1'}
    tau_list = np.arange(start,stop,step)
    size = mycamlogic.get_size()
    if odmr:
        odmrlogic.set_sweep_parameters(start, stop, step, power)
        odmrlogic.set_trigger(0, 1)
        odmrlogic.mw_sweep_on()
    else:
        set_start_MW(power, freq)
    save_array = np.zeros((len(tau_list), 2, size[1], size[0]), dtype = np.float64)
    for idx, t_ns in enumerate(tqdm(tau_list)):
        err = upload_CW_ensemble(t_ns, rep, pulse_fnt, rabi_ns, odmr)
        if err==-1:
            print('Memory error')
            pulsedmeasurementlogic.microwave_off()
            mycamera.pulsed_done()
            break
        img = np.zeros((2, size[1], size[0]), dtype = np.float64)
        for i in range(n_runs):
            img += do_cam_CW_pulsed_measure()
        img = img/n_runs
        save_array[idx] = img
    timestamp = datetime.datetime.now()
    t = timestamp.strftime("%Y%m%d-%H%M-%S")
    parameters = {
        'Tag': tag,
        'Start_ns': start,
        'Step_ns': step,
        'Stop_ns': stop,
        'ROI Size(px)': size,
        'No. of averaging runs': n_runs,
        'Repetitions': rep,
        'Pulse function': pulse_fnt.__name__,
        'Rabi period(ns)': rabi_ns,
        'MW Freq': freq,
        'MW power': power,
        'ODMR': odmr
    }
    data = {
        'Tau list (ns)': tau_list
    }
    save.save_data(data, filepath=save.get_path_for_module('Pulsed'), parameters=parameters, filename=None, filelabel=tag, timestamp=timestamp, filetype='text', fmt='%.15e', delimiter='\t', plotfig=None)
    np.savez_compressed(f'{save.get_path_for_module("Pulsed")}/{t}_{tag}_{start}_{stop}_{step}_{n_runs}_{size}pxs_{power}dBm', data=save_array)
    if odmr:
        odmrlogic.reset_sweep()
    pulsedmeasurementlogic.microwave_off()
    mycamera.pulsed_done()

### Pulsed ODMR

In [75]:
main(start=2.83e9, stop=2.87e9, step=500e3, n_runs=1, rep=10000, power=-16, freq=None, tag='odmr_small_roi', pulse_fnt=make_odmr_pulse_block, rabi_ns=(1/4073232)*1e9, odmr=True)

100%|######################################################################################################################################################################################################| 80/80 [09:28<00:00,  7.11s/it]


### Rabi

In [11]:
main(start=2010, stop=0, step=-25, n_runs=50, rep=30000, power=-20, freq=2.852e9, tag='rabi_ROI2', pulse_fnt=make_rabi_pulse_block, rabi_ns=None)

100%|####################################################################################################################################################################################################| 81/81 [1:33:50<00:00, 69.51s/it]


### T1

In [10]:
main(start=4501000, stop=0, step=-500000, n_runs=25, rep=750, power=-16, freq=2.852e9, tag='T1_ROI2', pulse_fnt=make_T1_pulse_block, rabi_ns=(1/1437424)*1e9)

  0%|                                                                                        | 0/10 [00:00<?, ?it/s]

here
here


 20%|###############8                                                               | 2/10 [12:40<52:01, 390.18s/it]

here


 30%|#######################7                                                       | 3/10 [17:11<41:22, 354.67s/it]

...

 80%|###############################################################2               | 8/10 [29:03<05:13, 156.96s/it]

here


 90%|#######################################################################1       | 9/10 [29:40<02:01, 121.17s/it]

here


100%|##############################################################################| 10/10 [29:53<00:00, 179.38s/it]


### HEcho

In [7]:
main(start=256000, stop=0, step=-10000, n_runs=50, rep=5000, power=-2, freq=2.8625e9, tag='T2_small_roi_6.91e6Hz', pulse_fnt=make_HEcho_pulse_block, rabi_ns=(1/6.910936e6)*1e9)

100%|###################################################################################################################################################################################################| 26/26 [1:03:16<00:00, 146.01s/it]


In [ ]:
# main(start=201000, stop=0, step=-5000, n_runs=100, rep=1000, power=-2, freq=2.8625e9, tag='T2_small_roi_6.91e6Hz_wref_pih', pulse_fnt=make_HEcho_pulse_block_w_ref_pih, rabi_ns=(1/6.910936e6)*1e9)
# main(start=201000, stop=0, step=-5000, n_runs=100, rep=1000, power=-2, freq=2.8625e9, tag='T2_small_roi_6.91e6Hz_wref_3pih', pulse_fnt=make_HEcho_pulse_block_w_ref_pih, rabi_ns=(1/6.910936e6)*1e9)

In [13]:
size = mycamlogic.get_size()
print(np.arange(4501000,0,-500000))
print(26+(4.5*250)+100+26+(4.5*250))

[4501000 4001000 3501000 3001000 2501000 2001000 1501000 1001000  501000
    1000]
2402.0
